In [0]:
%run "../../../FUNCOES/Classes_1.1"

In [0]:
end   = date.today()

try:
  start = (spark.read.table(f"mawe_gold.CONTRATOS_CANCELADOS").agg(max('DATA_ATUALIZACAO')).collect()[0][0]).strftime("%Y%m%d")
  start = start + timedelta(-15)
  d1 = True
except:
  start = end + timedelta(-10000)
  d1 = False

In [0]:
tipo_cancelamentos = ['ESTORNOS_VARIOS','DEVOLUCOES','CR:_RESSARCIMENTO','CR:_CREDITO_DEV','CHEQUES_DEVOLVIDOS','PERDAS_CONTAS_INCOB','DOCUMENTO_DO_CLIENTE']

In [0]:
contratos_alterados = spark.read.table(f"mawe_silver.PARTIDAS_COMPENSADAS").where(((col('DATA_ENTRADA') >= start) & (col('DATA_ENTRADA') <= end))|((col('DATA_COMPENSACAO') >= start) & (col('DATA_COMPENSACAO') <= end)))\
                                                                           .where(col('TIPO_DOCUMENTO').isin(tipo_cancelamentos))\
                                                                           .where(col('CONTRATO').isNotNull()).where(col('CONTRATO')!='').select('CONTRATO').drop_duplicates()

In [0]:
bsad = spark.read.table(f"mawe_silver.PARTIDAS_COMPENSADAS")

In [0]:
cli_cancelado = bsad.where((col('CONTRATO').isNotNull())&(col('ID_CLIENTE').isNotNull())).select('ID_CLIENTE','CONTRATO','DATA_COMPENSACAO')

In [0]:
bsad = bsad.where(col('CONDICAO_PAGAMENTO')!='PAGAMENTO_IMEDIATO_A_VISTA')

In [0]:
window  = Window.partitionBy(['EMPRESA','ID_CLIENTE','CONTRATO','CONDICAO_PAGAMENTO','TIPO_DOCUMENTO','DATA_BASE_VENCIMENTO']).orderBy(col('DATA_EXTRACAO').desc())
bsad = bsad.withColumn('tiebreak', monotonically_increasing_id()).withColumn('rank', rank().over(window)).filter(col('rank') == 1).drop('rank','tiebreak').drop_duplicates(subset=['EMPRESA','ID_CLIENTE','CONTRATO','CONDICAO_PAGAMENTO','TIPO_DOCUMENTO','DATA_BASE_VENCIMENTO'])

bsad = bsad.where(col('CONTRATO')!='')

In [0]:
bkpf = spark.read.table(f"mawe_silver.DOCUMENTOS_CONTABEIS")

In [0]:
bkpf_estorno = bkpf.select('EMPRESA','DATA_LANCAMENTO','DOCUMENTO_CONTABIL','TIPO_DOCUMENTO','DOCUMENTO_ESTORNO').withColumnRenamed('DATA_LANCAMENTO','DATA_COMPENSACAO').withColumnRenamed('DOCUMENTO_CONTABIL','DOCUMENTO_COMPENSACAO')
bkpf_estorno = bkpf_estorno.withColumn('ESTORNO', when(col('DOCUMENTO_ESTORNO').isNotNull(),lit(1)).when(col('TIPO_DOCUMENTO').isin(['DOCUMENTO_DO_CLIENTE','ESTORNOS_VARIOS']),lit(1)).otherwise(lit(0)))
bkpf_estorno = bkpf_estorno.select('EMPRESA','DATA_COMPENSACAO','DOCUMENTO_COMPENSACAO','ESTORNO')
bkpf_estorno = bkpf_estorno.where(col('ESTORNO')==1)

In [0]:
bkpf_devolucao = bkpf.select('EMPRESA','DATA_LANCAMENTO','DOCUMENTO_CONTABIL','TIPO_DOCUMENTO','DOCUMENTO_ESTORNO').withColumnRenamed('DATA_LANCAMENTO','DATA_COMPENSACAO').withColumnRenamed('DOCUMENTO_CONTABIL','DOCUMENTO_COMPENSACAO')
bkpf_devolucao = bkpf_devolucao.withColumn('ESTORNO', when(col('TIPO_DOCUMENTO').isin(['DEVOLUCOES']),lit(1)).otherwise(lit(0)))
bkpf_devolucao = bkpf_devolucao.select('EMPRESA','DATA_COMPENSACAO','DOCUMENTO_COMPENSACAO','ESTORNO')
bkpf_devolucao = bkpf_devolucao.where(col('ESTORNO')==1)

In [0]:
# para identificar perdas que foram pagas ou repactuadas para não ir para cancelamento

bkpf_perda = bkpf.select('EMPRESA','DATA_LANCAMENTO','DOCUMENTO_CONTABIL','TIPO_DOCUMENTO','DOCUMENTO_ESTORNO').withColumnRenamed('DATA_LANCAMENTO','DATA_COMPENSACAO').withColumnRenamed('DOCUMENTO_CONTABIL','DOCUMENTO_COMPENSACAO')
bkpf_perda = bkpf_perda.withColumn('PERDA_BKPF', when(col('TIPO_DOCUMENTO').isin(['PAGAMENTO_CLIENTE','REPACTUACAO','DOCUMENTO_CONTABIL']),lit(1)).otherwise(lit(0)))
bkpf_perda = bkpf_perda.select('EMPRESA','DATA_COMPENSACAO','DOCUMENTO_COMPENSACAO','PERDA_BKPF')
bkpf_perda = bkpf_perda.where(col('PERDA_BKPF')==1)

# Estornos

In [0]:
# SEPARAR OS ESTORNOS
estornos = bsad.where(col('DEBITO_CREDITO')=='DEBITO').where(col('TIPO_DOCUMENTO')=='ESTORNOS_VARIOS')
# RETORNAR TODAS AS LINHAS PELO DOCUMENTO DE COMPENSACAO, CLIENTE E EMPRESA
estornos = estornos.select(['ID_CLIENTE','EMPRESA','DOCUMENTO_COMPENSACAO'])
estornos = estornos.drop_duplicates()
estornos = estornos.join(bsad,on=['ID_CLIENTE','EMPRESA','DOCUMENTO_COMPENSACAO'],how='left')
# CHECK NOS ESTORNOS
estornos = estornos.select(['ID_CLIENTE','EMPRESA','DOCUMENTO_CONTABIL'])
estornos = estornos.withColumnRenamed('DOCUMENTO_CONTABIL', 'DOCUMENTO_COMPENSACAO')
estornos = estornos.withColumn('ESTORNO',lit(1))
cancelados_estornos = bsad.join(estornos,on=['ID_CLIENTE','EMPRESA','DOCUMENTO_COMPENSACAO'],how='left').na.fill({'ESTORNO':0})
cancelados_estornos = cancelados_estornos.groupby(['ID_CLIENTE','CONTRATO']).agg(max('DATA_COMPENSACAO').alias('DATA_CANCELAMENTO'),min('ESTORNO').alias('CANCELADO'))
cancelados_estornos = cancelados_estornos.where(col('CANCELADO')==1).drop('CANCELADO')

In [0]:
cancelados_bkpf_estorno = bsad.where(col('DEBITO_CREDITO')=='DEBITO').join(bkpf_estorno,on=['EMPRESA','DATA_COMPENSACAO','DOCUMENTO_COMPENSACAO'],how='left').na.fill({'ESTORNO':0})
cancelados_bkpf_estorno = cancelados_bkpf_estorno.groupby(['ID_CLIENTE','CONTRATO']).agg(max('DATA_COMPENSACAO').alias('DATA_CANCELAMENTO'),min('ESTORNO').alias('CANCELADO'))
cancelados_bkpf_estorno = cancelados_bkpf_estorno.where(col('CANCELADO')==1).drop('CANCELADO')

In [0]:
cancelados_bkpf_devolucao = bsad.where(col('DEBITO_CREDITO')=='DEBITO').join(bkpf_devolucao,on=['EMPRESA','DATA_COMPENSACAO','DOCUMENTO_COMPENSACAO'],how='left').na.fill({'ESTORNO':0})
cancelados_bkpf_devolucao = cancelados_bkpf_devolucao.groupby(['ID_CLIENTE','CONTRATO']).agg(max('DATA_COMPENSACAO').alias('DATA_CANCELAMENTO'),max('ESTORNO').alias('CANCELADO'))
cancelados_bkpf_devolucao = cancelados_bkpf_devolucao.where(col('CANCELADO')==1).drop('CANCELADO')

# Devoluções

In [0]:
devolucoes = bsad.where(col('TIPO_DOCUMENTO')=='DEVOLUCOES')

# RETORNAR TODAS AS LINHAS PELO DOCUMENTO DE COMPENSACAO, CLIENTE E EMPRESA
devolucoes = devolucoes.select(['ID_CLIENTE','EMPRESA','DOCUMENTO_COMPENSACAO'])
devolucoes = devolucoes.drop_duplicates()
devolucoes = devolucoes.join(bsad,on=['ID_CLIENTE','EMPRESA','DOCUMENTO_COMPENSACAO'],how='left')

devolucoes = devolucoes.select(['ID_CLIENTE','EMPRESA','DOCUMENTO_CONTABIL'])
devolucoes = devolucoes.withColumnRenamed('DOCUMENTO_CONTABIL', 'DOCUMENTO_COMPENSACAO')
devolucoes = devolucoes.withColumn('DEVOLUCAO',lit(1))
cancelados_devolucoes = bsad.join(devolucoes,on=['ID_CLIENTE','EMPRESA','DOCUMENTO_COMPENSACAO'],how='left').na.fill({'DEVOLUCAO':0})
cancelados_devolucoes = cancelados_devolucoes.groupby(['ID_CLIENTE','CONTRATO']).agg(max('DATA_COMPENSACAO').alias('DATA_CANCELAMENTO'),max('DEVOLUCAO').alias('CANCELADO'))
cancelados_devolucoes = cancelados_devolucoes.where(col('CANCELADO')==1).drop('CANCELADO')
cancelados_devolucoes = cancelados_devolucoes.where(col('CONTRATO').isNotNull())

# Ressarcimentos

In [0]:
ressarcimentos = bsad.where((col('TIPO_DOCUMENTO')=='CR:_RESSARCIMENTO')|(col('TIPO_DOCUMENTO')=='CR:_CREDITO_DEV')|(col('TIPO_DOCUMENTO')=='CHEQUES_DEVOLVIDOS'))
# RETORNAR TODAS AS LINHAS PELO DOCUMENTO DE COMPENSACAO, CLIENTE E EMPRESA
ressarcimentos = ressarcimentos.select(['ID_CLIENTE','EMPRESA','DOCUMENTO_COMPENSACAO'])
ressarcimentos = ressarcimentos.drop_duplicates()
ressarcimentos = ressarcimentos.join(bsad,on=['ID_CLIENTE','EMPRESA','DOCUMENTO_COMPENSACAO'],how='left')
# CHECK NOS ESTORNOS
ressarcimentos = ressarcimentos.select(['ID_CLIENTE','EMPRESA','DOCUMENTO_CONTABIL'])
ressarcimentos = ressarcimentos.withColumnRenamed('DOCUMENTO_CONTABIL', 'DOCUMENTO_COMPENSACAO')
ressarcimentos = ressarcimentos.withColumn('DEVOLUCAO',lit(1))
cancelados_ressarcimentos = bsad.join(ressarcimentos,on=['ID_CLIENTE','EMPRESA','DOCUMENTO_COMPENSACAO'],how='left').na.fill({'DEVOLUCAO':0})
cancelados_ressarcimentos = cancelados_ressarcimentos.groupby(['ID_CLIENTE','CONTRATO']).agg(max('DATA_COMPENSACAO').alias('DATA_CANCELAMENTO'),max('DEVOLUCAO').alias('CANCELADO'))
cancelados_ressarcimentos = cancelados_ressarcimentos.where(col('CANCELADO')==1).drop('CANCELADO')
cancelados_ressarcimentos = cancelados_ressarcimentos.where(col('CONTRATO').isNotNull())

# Perdas

In [0]:
# SEPARAR OS ESTORNOS
estornos = bsad.where(col('DEBITO_CREDITO')=='DEBITO').where(col('TIPO_DOCUMENTO')=='ESTORNOS_VARIOS')
# RETORNAR TODAS AS LINHAS PELO DOCUMENTO DE COMPENSACAO, CLIENTE E EMPRESA
estornos = estornos.select(['ID_CLIENTE','EMPRESA','DOCUMENTO_COMPENSACAO'])
estornos = estornos.drop_duplicates()
estornos = estornos.join(bsad,on=['ID_CLIENTE','EMPRESA','DOCUMENTO_COMPENSACAO'],how='left')
# CHECK NOS ESTORNOS
estornos = estornos.select(['ID_CLIENTE','EMPRESA','DOCUMENTO_CONTABIL'])
estornos = estornos.withColumnRenamed('DOCUMENTO_CONTABIL', 'DOCUMENTO_COMPENSACAO')
estornos = estornos.withColumn('ESTORNO',lit(1))
bsad = bsad.join(estornos,on=['ID_CLIENTE','EMPRESA','DOCUMENTO_COMPENSACAO'],how='left').na.fill({'ESTORNO':0})
bsad = bsad.where(col('ESTORNO')!=1)

In [0]:
perdas = bsad.where((col('TIPO_DOCUMENTO')=='PERDAS_CONTAS_INCOB'))
# RETORNAR TODAS AS LINHAS PELO DOCUMENTO DE COMPENSACAO, CLIENTE E EMPRESA
perdas = perdas.select(['ID_CLIENTE','EMPRESA','DOCUMENTO_COMPENSACAO'])
perdas = perdas.drop_duplicates()
perdas = perdas.join(bsad,on=['ID_CLIENTE','EMPRESA','DOCUMENTO_COMPENSACAO'],how='left')
# CHECK NOS ESTORNOS
perdas = perdas.select(['ID_CLIENTE','EMPRESA','DOCUMENTO_CONTABIL'])
perdas = perdas.withColumnRenamed('DOCUMENTO_CONTABIL', 'DOCUMENTO_COMPENSACAO')
perdas = perdas.withColumn('PERDAS',lit(1))
cancelados_perdas = bsad.join(perdas,on=['ID_CLIENTE','EMPRESA','DOCUMENTO_COMPENSACAO'],how='left').na.fill({'PERDAS':0})

In [0]:
cancelados_perdas = cancelados_perdas.where(col('DEBITO_CREDITO')=='DEBITO').join(bkpf_perda,on=['EMPRESA','DATA_COMPENSACAO','DOCUMENTO_COMPENSACAO'],how='left').na.fill({'PERDA_BKPF':0})

In [0]:
# é perda de fato quando tem PCI na bsad (PERDAS==1) e, pela bkpf, essa perda não é um pagamento, repactuação ou documento contábil (PERDA_BKPF==0)
cancelados_perdas = cancelados_perdas.where((col('PERDAS')==1)&(col('PERDA_BKPF')==0))

In [0]:
cancelados_perdas = cancelados_perdas.groupby(['ID_CLIENTE','CONTRATO']).agg(max('DATA_COMPENSACAO').alias('DATA_CANCELAMENTO'))
cancelados_perdas = cancelados_perdas.where(col('CONTRATO').isNotNull())

# CLIs errados

In [0]:
def checar_cli(cliente, contrato):
  try:
    cliente = int(cliente)
    contrato = int(contrato[3:][:-3])
    if cliente!=contrato:
      return 1
    else:
      return 0
  except:
    return 1
  
  
checar_cli = udf(checar_cli)

In [0]:
cli_cancelado = cli_cancelado.withColumn('CANCELADO', checar_cli(col('ID_CLIENTE'),col('CONTRATO')))
cli_cancelado = cli_cancelado.where(col('CANCELADO')=='1')
cli_cancelado = cli_cancelado.groupby(['ID_CLIENTE','CONTRATO']).agg(max('DATA_COMPENSACAO').alias('DATA_CANCELAMENTO'))

In [0]:
cli_cancelado2 = spark.read.table(f"mawe_gold.CONTRATOS").withColumn('CANCELADO', checar_cli(col('ID_CLIENTE'),col('CONTRATO')))
cli_cancelado2 = cli_cancelado2.where(col('CANCELADO')=='1')
cli_cancelado2 = cli_cancelado2.groupby(['ID_CLIENTE','CONTRATO']).agg(max('DATA_CONTRATO').alias('DATA_CANCELAMENTO'))

In [0]:
cancelados = cancelados_estornos.union(cancelados_devolucoes).drop_duplicates(subset=['ID_CLIENTE','CONTRATO'])
cancelados = cancelados.union(cancelados_ressarcimentos).drop_duplicates(subset=['ID_CLIENTE','CONTRATO'])
cancelados = cancelados.union(cancelados_bkpf_estorno).drop_duplicates(subset=['ID_CLIENTE','CONTRATO'])
cancelados = cancelados.union(cancelados_bkpf_devolucao).drop_duplicates(subset=['ID_CLIENTE','CONTRATO'])
cancelados = cancelados.union(cancelados_ressarcimentos).drop_duplicates(subset=['ID_CLIENTE','CONTRATO'])
cancelados = cancelados.union(cancelados_perdas).drop_duplicates(subset=['ID_CLIENTE','CONTRATO'])
cancelados = cancelados.union(cli_cancelado).drop_duplicates(subset=['ID_CLIENTE','CONTRATO'])
cancelados = cancelados.union(cli_cancelado2).drop_duplicates(subset=['ID_CLIENTE','CONTRATO'])

In [0]:
if d1==True: cancelados = cancelados.join(contratos_alterados,on='CONTRATO',how='inner')

In [0]:
key_columns = ['ID_CLIENTE','CONTRATO','DATA_CANCELAMENTO']
bk_columns_keys = [coluna + '_DEL' for coluna in key_columns]

for coluna in key_columns:
  cancelados = cancelados.withColumn(coluna + '_DEL', col(coluna).cast(StringType()))

In [0]:
cancelados = cancelados.withColumn("BK_EXTRACAO", F.md5(F.concat_ws('_', array(bk_columns_keys))))

In [0]:
colunas = cancelados.columns

for coluna in colunas:
  if '_DEL' in coluna:
    cancelados = cancelados.drop(coluna) 

In [0]:
cancelados = cancelados.withColumn('DATA_ATUALIZACAO',date_add(current_date(),-1))

In [0]:
cancelados = cancelados.withColumn("MES_ANO", date_format('DATA_ATUALIZACAO','yyyyMM').cast(StringType()))

In [0]:
###### Monta variaveis da Delta Table
schema_gold  = "mawe_gold"
gold_table = "CONTRATOS_CANCELADOS"
key_columns  = ["BK_EXTRACAO"]
partition    = ["MES_ANO"]

###### busca caminho delta table na camada
location = Framework.generate_gold_table_location(
    schema_name=schema_gold, 
    table_name=gold_table,
)

In [0]:
## executa a gravação da tabela gold
results = Framework.write_delta_table(
    df=cancelados,
    location=location,
    schema_name=schema_gold,
    table_name=gold_table,
    load_type=Framework.LoadType.UPSERT,
    key_columns=key_columns,
    partition_columns=partition,
   # schema_evolution_mode=Framework.SchemaEvolutionMode.ADD_NEW_COLUMNS,
)